[Website](https://finance.yahoo.com/markets/stocks/most-active/)

In [7]:
import requests
import pandas as pd 
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [8]:
stock = []

path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.set_page_load_timeout(120)

try: 
    url = 'https://finance.yahoo.com/markets/stocks/most-active/?start=0&count=25' 
    driver.get(url) 

    for _ in range(10): 
        time.sleep(5)

        pager = driver.find_elements(By.TAG_NAME, 'tbody')

        for page in pager:
            rows = page.find_elements(By.TAG_NAME, 'tr')

            for row in rows:
                try:
                    data = row.find_elements(By.TAG_NAME, 'td')

                    row_data = {
                            "Symbol": data[0].text.strip(),
                            "Name": data[1].text.strip(),
                            "Price": data[3].text.strip(),
                            "Change": data[4].text.strip(),
                            "Change %": data[5].text.strip(),
                            "Volume": data[6].text.strip(),
                            "Avg Vol (3M)": data[7].text.strip(),
                            "Market Cap": data[8].text.strip(),
                            "P/E Ratio (TTM)": data[9].text.strip(),
                            "52 Wk Change %": data[10].text.strip()
                        }
                    stock.append(row_data) 

                except Exception as e:
                        print(f"Error parsing table: {e}")
                
        next_button = WebDriverWait(driver,10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='next-page-button']"))
        )
        next_button.click()

        time.sleep(5)
except:
    pass

finally:
     driver.quit()
            

In [9]:
j_path = "Most Active Stocks.json"

with open(j_path, 'w') as file:
    json.dump(stock, file, indent=4)

print(f"Data successfully saved to {j_path}")

Data successfully saved to Most Active Stocks.json


In [10]:
df = pd.DataFrame(stock)
df.to_csv('Most Active Stocks.csv')